In [1]:
from datetime import datetime
from pprint import pprint
import re
import sqlite3

In [2]:
db = sqlite3.connect('database.sqlite3')

In [3]:
!sqlite3 database.sqlite3 .schema

-- Loading resources from /home/remram/.sqliterc
CREATE TABLE windows(
                id INTEGER PRIMARY KEY,
                start DATETIME NOT NULL,
                end DATETIME NOT NULL,
                active BOOLEAN NOT NULL,
                name TEXT NOT NULL
            );
CREATE TABLE runs(
                id INTEGER PRIMARY KEY,
                start DATETIME NOT NULL,
                end DATETIME NULL,
                end_reason TEXT NOT NULL DEFAULT ''
            );
CREATE INDEX idx_windows_start ON windows(start);
CREATE INDEX idx_windows_end ON windows(end);
CREATE INDEX idx_windows_active ON windows(active) WHERE active=1;
CREATE INDEX idx_windows_name ON windows(name);
CREATE INDEX idx_runs_start ON runs(start);
CREATE INDEX idx_runs_end ON runs(end);


In [4]:
list(db.execute('SELECT count(id) FROM windows;'))

[(2111,)]

In [5]:
names = [name for name, in db.execute('SELECT DISTINCT name FROM windows WHERE active=1;')]
len(names)

80

In [6]:
for name in sorted(names):
    print(name)

() datamart-prod — Konsole
(NSFW) What Fetish did you have until you tried it? : AskReddit — Mozilla Firefox
- : sudo -g — Konsole
/home/remram/projects/pctracker - QGit
2017 IRONMAN Lake Placid » RTRT.me — Mozilla Firefox
ASMR Cranial Nerve Exam: Concussion Test 👩‍⚕️ (Soft Spoken Roleplay - Doctor Check Up ASMR) - YouTube — Mozilla Firefox
Add Comment | Hacker News — Mozilla Firefox
Beer distribution game - Wikipedia — Mozilla Firefox
Bevy ECS migration by cart · Pull Request #49 · bevyengine/bevy — Mozilla Firefox
Billing Statement - remirampin@gmail.com - Gmail — Mozilla Firefox
Blue Jeans Network | Video Collaboration in the Cloud - Google Chrome
CGi Artist Makes a Masterpiece Everyday For 12 Years (Today We Challenge Him) - YouTube — Mozilla Firefox
Capital One | Bank Details — Mozilla Firefox
Capture : bash — Konsole
Cerulean: Experimental micro-blogging app for Matrix | Hacker News — Mozilla Firefox
Copy of GIS_2020-12-15_Gmail - 4et50yp2ckm8otv.pdf — Mozilla Firefox
DEF CON 22 

In [7]:
def regex(pattern, name=None):
    compiled = re.compile(pattern)
    
    def regex_matcher(record):
        return compiled.search(record.name) is not None
    
    if not name:
        name = repr(pattern)
    
    return name, regex_matcher

In [8]:
filters = [
    (regex(' — Mozilla Firefox$', 'Firefox'), [
        (regex(' - YouTube — Mozilla Firefox$', 'YouTube'), []),
        (regex(' Hacker News — Mozilla Firefox$', 'HackerNews'), []),
        (regex(' GitLab — Mozilla Firefox$', 'GitLab'), []),
        (regex(' - Gmail — Mozilla Firefox$', 'Gmail'), []),
        (regex('Slack .* — Mozilla Firefox$', 'Slack'), []),
        (regex(' - Jupyter Notebook — Mozilla Firefox$', 'Jupyter'), []),
    ]),
    (regex(' - Google Chrome$', 'Google Chrome'), []),
    (regex('^Signal', 'Signal'), []),
    (regex(' - GVIM$', 'GVIM'), []),
    (regex(' — Konsole$', 'Konsole'), []),
]

In [9]:
class Record(object):
    def __init__(self, start, end, name):
        self.start = datetime.fromisoformat(start)
        self.end = datetime.fromisoformat(end)
        self.name = name
        
    @property
    def duration(self):
        return (self.end - self.start).total_seconds()

In [10]:
def format_duration(seconds):
    seconds = int(seconds)
    out = '%ds' % (seconds % 60)
    if seconds >= 60:
        out = '%dm%s' % ((seconds // 60) % 60, out)
        if seconds >= 3600:
            out = '%dh%s' % (seconds // 3600, out)
    return out

In [11]:
class OutputNode(object):
    def __init__(self, name):
        self.name = name
        self.duration = 0
        self.children = {}
        
    def child(self, name):
        try:
            child = self.children[name]
        except KeyError:
            child = self.children[name] = OutputNode(name)
        return child
    
    def print(self, indent=0):
        if indent == 0:
            prefix = ''
            last_prefix = '└── '
        else:
            prefix = '│   ' * (indent - 1)
            prefix += '├── '
            last_prefix = '|   ' * indent
            last_prefix += '└── '
        name = self.name or 'total'
        other_duration = self.duration
        duration = format_duration(self.duration)
        print(f'{prefix}{name} {duration}')
        children = sorted(self.children.values(), key=lambda c: -c.duration)
        for child in children:
            child.print(indent + 1)
            other_duration -= child.duration
        if self.children:
            print(f'{last_prefix}other {other_duration}')

In [12]:
def apply_filters(record, filters, output):
    # Add duration to current node
    output.duration += record.duration
    # Apply filters until one matches
    for (name, matcher), then_ops in filters:
        if matcher(record):
            apply_filters(record, then_ops, output.child(name))
            return


output = OutputNode(None)

for row in db.execute('''\
    SELECT start, end, name
    FROM windows
    WHERE active=1;
'''):
    record = Record(*row)
    apply_filters(record, filters, output)

In [13]:
output.print()

total 2h9m12s
├── Firefox 1h40m3s
│   ├── YouTube 19m26s
│   ├── Gmail 9m48s
│   ├── Jupyter 7m3s
│   ├── GitLab 5m17s
│   ├── Slack 5m7s
│   ├── HackerNews 2m43s
|   └── other 3039.0
├── Konsole 11m4s
├── Signal 3m43s
├── GVIM 1m26s
├── Google Chrome 26s
└── other 750.0
